In [36]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

selected_features = ["duration_ms_x","danceability","energy", "instrumentalness", "liveness", "valence", "speechiness", "tempo"] 

SUCCESS_FEATURE = 'popularity'
SUCCESS_THRESHOLD = 0.75
MILD_SUCCESS_THRESHOLD = 0.4

df = pd.read_csv("chaabi.csv")
df = df.sort_values(by=['popularity'])
df = df.drop_duplicates(subset=['id'])
df = df.drop_duplicates(subset=['name', 'main_artist'], keep='first')
df = df.drop_duplicates(subset=selected_features, keep='first')


In [37]:
df.describe()

,key,mode,time_signature,duration_ms_x,danceability,loudness,energy,instrumentalness,liveness,valence,speechiness,tempo,popularity,youtube_views
count,958.000000,958.000000,958.000000,9.580000e+02,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,9.580000e+02
mean,5.075157,0.581420,3.757829,4.602864e+05,0.517825,-8.917685,0.601087,0.120644,0.244710,0.775332,0.058222,119.205842,2.834029,2.060257e+05
std,3.628891,0.493584,0.504749,4.431472e+05,0.117708,3.707884,0.185415,0.267462,0.185106,0.156906,0.035857,25.539410,4.714482,8.718157e+05
min,0.000000,0.000000,1.000000,6.998400e+04,0.227000,-27.767000,0.000246,0.000000,0.021600,0.140000,0.026900,50.275000,0.000000,0.000000e+00
25%,2.000000,0.000000,3.000000,2.333898e+05,0.427000,-10.720500,0.466000,0.000016,0.113000,0.692000,0.038100,102.158750,0.000000,3.305000e+02
50%,5.000000,1.000000,4.000000,3.400080e+05,0.517500,-8.720000,0.587000,0.000493,0.180500,0.812000,0.047050,113.075000,1.000000,3.655000e+03
75%,8.000000,1.000000,4.000000,4.749628e+05,0.602000,-6.496750,0.736750,0.030150,0.333000,0.897750,0.064425,136.457000,4.000000,4.177175e+04
max,11.000000,1.000000,5.000000,4.388522e+06,0.850000,2.917000,0.983000,0.973000,0.952000,0.993000,0.421000,209.148000,35.000000,1.432023e+07


## Feature engineering

In [38]:
df['popularity_cat'] = "Unsuccesful"
# TODO Use spotify, Youtube and Last FM 
df.loc[(df[SUCCESS_FEATURE] > df[SUCCESS_FEATURE].quantile(SUCCESS_THRESHOLD)), 'popularity_cat'] = "Succesful"
df.loc[(df[SUCCESS_FEATURE] <= df[SUCCESS_FEATURE].quantile(SUCCESS_THRESHOLD)) & (df[SUCCESS_FEATURE] > df[SUCCESS_FEATURE].quantile(MILD_SUCCESS_THRESHOLD)), 'popularity_cat'] = "Mildly Succesful"

## Split & prepare the data

In [39]:
train, test = train_test_split(df, test_size=0.2)

selected_target = "popularity_cat"

train_x = train[selected_features] 
train_y = train[selected_target]

test_x = test[selected_features] 
test_y = test[selected_target]

## Building the model

In [40]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.dummy import DummyClassifier
forest = RandomForestClassifier(max_depth=5, min_samples_leaf=2, n_estimators=50)
dummy_clf = DummyClassifier(strategy='stratified')

In [41]:
forest.fit(X = train_x, y = train_y)
dummy_clf.fit(X = train_x, y = train_y)

DummyClassifier(strategy='stratified')

In [42]:
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
rf_predict = forest.predict(test_x)
train_predict = forest.predict(train_x)
dummy_predict = dummy_clf.predict(test_x)
print("Training accuracy: " + str(accuracy_score(train_y, train_predict)))
print("Testing accuracy: " + str(accuracy_score(test_y, rf_predict)))
print("Dummy accuracy: " + str(accuracy_score(test_y, dummy_predict)))

Training accuracy: 0.6449086161879896
Testing accuracy: 0.5572916666666666
Dummy accuracy: 0.4270833333333333


In [43]:
for name, score in zip(selected_features, forest.feature_importances_):
    print(name, score)

duration_ms_x 0.16451103956462318
danceability 0.09814895736717424
energy 0.15171401081081323
instrumentalness 0.1094485246640321
liveness 0.06986313547785448
valence 0.1022936143761152
speechiness 0.20009168781755254
tempo 0.10392902992183509


In [44]:
test["prediction"] = rf_predict
test.sample(5)

,id,name,main_artist,key,mode,time_signature,duration_ms_x,danceability,loudness,energy,instrumentalness,liveness,valence,speechiness,tempo,popularity,youtube_views,popularity_cat,prediction
511,6jlN6txUOtro1Ir6eRVEQw,Klam El Aare,Naima Dziria,0,1,4,164362,0.320,-8.876,0.952,0.000001,0.706,0.858,0.0645,155.429,3,105,Mildly Succesful,Mildly Succesful
174,3rLn08zyJW0nCwodk9dxFU,Ouryenfagh manahder - Kabyle,Kamel Messaoudi,10,1,4,442524,0.660,-8.943,0.626,0.000426,0.131,0.726,0.1120,136.461,10,735865,Succesful,Unsuccesful
536,7hVzCDmnxC0EBFbjKLMyWB,Awah ya dmaghi,El Hachemi Guerouabi,11,1,4,662835,0.497,-11.969,0.331,0.064600,0.124,0.674,0.0636,119.234,2,111221,Mildly Succesful,Unsuccesful
1061,1LSc7wU36U9I2Jq30NRTYY,Youm el djemaâ,Cheikh el Hasnaoui,3,0,3,335987,0.444,-8.910,0.476,0.000000,0.369,0.671,0.0937,82.088,0,23254,Unsuccesful,Unsuccesful
1398,4kaLtYtExZPOBZ9FBpsifj,El Mektoub,El Hachemi Guerouabi,2,1,4,440293,0.467,-15.563,0.348,0.000092,0.246,0.666,0.0562,112.503,0,198451,Unsuccesful,Unsuccesful
